In [2]:
import torchxrayvision as xrv
import skimage, torch, torchvision
from pathlib import Path

In [3]:
xrv_224 = "densenet121-res224-all"
xrv_512 = "resnet50-res512-all"
xrv_224_chex ="densenet121-res224-chex"
model_xrv_224 = xrv.models.DenseNet(weights=xrv_224)
model_xrv_512 = xrv.models.ResNet(weights=xrv_512)
model_xrv_224_chex = xrv.models.DenseNet(weights=xrv_224_chex)

In [6]:
def process_image(model,image_path):
    # NOTE: this is the example processing code from the xrv readme
    img = skimage.io.imread(image_path)
    img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
    img = img[None,...] 


    if "224" in model:
        transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
    else:
        transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(512)])
    img = transform(img)
    img = torch.from_numpy(img).unsqueeze(0)

    return img

In [21]:
cheXpert_small_val_split = Path("/vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/valid.csv")
cheXpert_small_test_split = Path("/vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv")

cheXpert_small_validation_path = Path("/vol/biodata/data/chest_xray/CheXpert-v1.0-small")
cheXpert_small_test_path = Path("/vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/")

def generate_outputs(model_name, model, image_paths, test = False):

    output_dict = {}

    with open(image_paths) as f:
        f.readline() # skip header
        for i, line in enumerate(f):
            if i % 100 == 0:
                print(f"Processing image {i} of {image_paths}")
            img_path = line.split(",")[0]

            if test:
                img_path = cheXpert_small_test_path / img_path
            else:
                img_path = cheXpert_small_validation_path / img_path

            processed_image = process_image(model_name, img_path)
            model_output = model(processed_image)
            processed_outputs = [k for k,v in dict(zip(model_xrv_224_chex.pathologies,model_output[0].detach().numpy())).items() if v > 0.5]
            if len(processed_outputs) == 0:
                processed_outputs = ["No findings"]
            key = ("/").join(str(img_path).split("/")[-3:])
            output_dict[key] = processed_outputs

    return output_dict


In [22]:
out_dir = Path("/vol/biomedic3/bglocker/ugproj2324/nns20/torchxrayvision/evaluations/CheXpert_evaluation_results/test_set")
out_dir.mkdir(exist_ok=True,parents=True)
out_224_path = out_dir / 'xrv_224.txt'
out_224_chex_path = out_dir / 'xrv_224_chex.txt'
out_512_path = out_dir / 'xrv_512.txt'

out_224 = generate_outputs(xrv_224, model_xrv_224, cheXpert_small_test_split,test=True)
out_224_chex = generate_outputs(xrv_224_chex, model_xrv_224_chex, cheXpert_small_test_split,test=True)
out_512 = generate_outputs(xrv_512, model_xrv_512, cheXpert_small_test_split,test=True)

with open(out_224_path, 'w') as f:
    for img_id,outputs in out_224.items():
        f.write(f'{img_id},{",".join(outputs)}\n')

with open(out_224_chex_path, 'w') as f:
    for img_id,outputs in out_224_chex.items():
        f.write(f'{img_id},{",".join(outputs)}\n')

with open(out_512_path, 'w') as f:
    for img_id,outputs in out_512.items():
        f.write(f'{img_id},{",".join(outputs)}\n')


Processing image 0 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 100 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 200 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 300 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 400 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 500 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 600 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 0 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 100 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 200 of /vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test.csv
Processing image 300 of 